In [43]:
import pandas as pd
import numpy as np
#import findspark
#findspark.init()
from pyspark.sql import SparkSession , SQLContext
from pyspark import SparkContext,SparkConf
import plotly.graph_objects as go
import plotly.express as px

In [44]:
SRR_aMetagenomes=pd.read_table("Metadata/aMetagenomes/SRR_aMetagenomes.txt", names=["Accession"],)

In [45]:
aMetagenomes_SRStoSRR=pd.read_table("Metadata/aMetagenomes/aMetagenomes_SRStoSRR.tsv",names=["archive_accession","SRR_accession"])
aMetagenomes=aMetagenomes_SRStoSRR[aMetagenomes_SRStoSRR['SRR_accession'].notna()]
aMetagenomes_SRStoSRR

,archive_accession,SRR_accession
0,SRS473742,SRR957738
1,SRS473743,SRR957739
2,SRS473744,SRR957740
3,SRS473745,SRR957741
4,SRS473747,SRR957742
...,...,...
513,SRS1815433,SRR5047202
514,SRS1815336,SRR5047105
515,SRS1815334,SRR5047103
516,SRS1815373,SRR5047142


In [46]:
aMetagenomes_metadata=pd.read_table("Metadata/aMetagenomes/aMetagenomes.txt")
#Create one row per archive_accession
aMetagenomes_metadata=aMetagenomes_metadata.assign(archive_accession=aMetagenomes_metadata['archive_accession'].str.split(',')).explode('archive_accession')
aMetagenomes_metadata=aMetagenomes_metadata.reset_index()
aMetagenomes_metadata=aMetagenomes_metadata[aMetagenomes_metadata["archive"]=="SRA"]
aMetagenomes_metadata.shape

(447, 18)

In [47]:
aMetagenomes_metadata.head()

,index,project_name,publication_year,publication_doi,site_name,latitude,longitude,geo_loc_name,sample_name,sample_host,sample_age,sample_age_doi,community_type,material,collection_date,archive,archive_project,archive_accession
0,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473742
1,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473743
2,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473744
3,1,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,B61,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473745
4,2,Warinner2014,2014,10.1038/ng.2906,Dalheim,51.565,8.84,Germany,G12,Homo sapiens,900,10.1038/ng.2906,oral,dental calculus,NaN,SRA,PRJNA216965,SRS473747


In [48]:
len(aMetagenomes_SRStoSRR.SRR_accession)

518

In [49]:
aMetagenomes_metadata.shape

(447, 18)

In [50]:
#Filter aMetagenomes_metadata with only files that were downloaded

In [51]:
SRR_aMetagenomes=pd.read_table("Metadata/aMetagenomes/SRR_aMetagenomes.txt",names=["SRR_accession"])
SRR_aMetagenomes

,SRR_accession
0,SRR957738
1,SRR957739
2,SRR957740
3,SRR957741
4,SRR957742
...,...
442,SRR5047202
443,SRR5047105
444,SRR5047103
445,SRR5047142


In [52]:
a=pd.merge(aMetagenomes_SRStoSRR,SRR_aMetagenomes,on="SRR_accession",how="inner")
a

,archive_accession,SRR_accession
0,SRS473742,SRR957738
1,SRS473743,SRR957739
2,SRS473744,SRR957740
3,SRS473745,SRR957741
4,SRS473747,SRR957742
...,...,...
478,SRS1815433,SRR5047202
479,SRS1815336,SRR5047105
480,SRS1815334,SRR5047103
481,SRS1815373,SRR5047142


In [53]:
a.drop_duplicates()

,archive_accession,SRR_accession
0,SRS473742,SRR957738
1,SRS473743,SRR957739
2,SRS473744,SRR957740
3,SRS473745,SRR957741
4,SRS473747,SRR957742
...,...,...
478,SRS1815433,SRR5047202
479,SRS1815336,SRR5047105
480,SRS1815334,SRR5047103
481,SRS1815373,SRR5047142


In [54]:
fig = px.histogram(aMetagenomes_metadata, x="sample_age",title="Sample age")
fig.show()

In [55]:
fig = px.histogram(aMetagenomes_metadata, x="sample_host",title="Sample host")
fig.show()

In [56]:
fig = px.histogram(aMetagenomes_metadata, x="community_type",title="Community type")
fig.show()

In [57]:
fig = px.histogram(aMetagenomes_metadata, x="material",title="material")
fig.show()

In [58]:
fig = px.scatter_mapbox(aMetagenomes_metadata, lat="latitude", lon="longitude", hover_name="geo_loc_name",
                        hover_data=['site_name'],
                        color_discrete_sequence=["red"], zoom=2, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# PySPARK <3

In [62]:
# Define session
#The following line should be run if the last session was not closed
sc.stop()
conf = SparkConf().setAppName("First attempt").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [68]:
aMetagenomes=spark.read.csv("kmMatrices/aMetagenomes/whole_aMetagenomes_kmMatrix.txt",header=False, sep=" ")

In [69]:
aMetagenomes.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29',
 '_c30',
 '_c31',
 '_c32',
 '_c33',
 '_c34',
 '_c35',
 '_c36',
 '_c37',
 '_c38',
 '_c39',
 '_c40',
 '_c41',
 '_c42',
 '_c43',
 '_c44',
 '_c45',
 '_c46',
 '_c47',
 '_c48',
 '_c49',
 '_c50',
 '_c51',
 '_c52',
 '_c53',
 '_c54',
 '_c55',
 '_c56',
 '_c57',
 '_c58',
 '_c59',
 '_c60',
 '_c61',
 '_c62',
 '_c63',
 '_c64',
 '_c65',
 '_c66',
 '_c67',
 '_c68',
 '_c69',
 '_c70',
 '_c71',
 '_c72',
 '_c73',
 '_c74',
 '_c75',
 '_c76',
 '_c77',
 '_c78',
 '_c79',
 '_c80',
 '_c81',
 '_c82',
 '_c83',
 '_c84',
 '_c85',
 '_c86',
 '_c87',
 '_c88',
 '_c89',
 '_c90',
 '_c91',
 '_c92',
 '_c93',
 '_c94',
 '_c95',
 '_c96',
 '_c97',
 '_c98',
 '_c99',
 '_c100',
 '_c101',
 '_c102',
 '_c103',
 '_c104',
 '_c105',
 '_c106',
 '_c107',
 '_c108',
 '_c109',
 '_c110',


In [33]:
colnames=["K-mer"]+list(SRR_aMetagenomes.SRR_accession)

In [34]:
colnames

['K-mer',
 'SRR957738',
 'SRR957739',
 'SRR957740',
 'SRR957741',
 'SRR957742',
 'SRR957743',
 'SRR957744',
 'SRR957745',
 'SRR957746',
 'SRR6877286',
 'SRR6877284',
 'SRR6877290',
 'SRR6877288',
 'SRR6877292',
 'SRR6877323',
 'SRR6877304',
 'SRR6877305',
 'SRR6877392',
 'SRR6877308',
 'SRR6877303',
 'SRR6877372',
 'SRR6877370',
 'SRR6877376',
 'SRR6877374',
 'SRR6877378',
 'SRR6877388',
 'SRR6877390',
 'SRR6877384',
 'SRR6877386',
 'SRR6877382',
 'SRR6877354',
 'SRR6877356',
 'SRR6877350',
 'SRR6877352',
 'SRR6877358',
 'SRR6877367',
 'SRR6877365',
 'SRR6877363',
 'SRR6877361',
 'SRR6877369',
 'SRR6877338',
 'SRR6877336',
 'SRR6877334',
 'SRR6877332',
 'SRR6877330',
 'SRR6877342',
 'SRR6877344',
 'SRR6877346',
 'SRR6877348',
 'SRR6877340',
 'SRR6877395',
 'SRR6877397',
 'SRR6877399',
 'SRR6877401',
 'SRR6877403',
 'SRR6877294',
 'SRR6877328',
 'SRR6877339',
 'SRR6877296',
 'SRR6877300',
 'SRR6877316',
 'SRR6877318',
 'SRR6877320',
 'SRR6877322',
 'SRR6877313',
 'SRR6877312',
 'SRR6877

In [35]:
#Change column names to dataframe
aMetagenomes=aMetagenomes.toDF(*colnames)

IllegalArgumentException: requirement failed: The number of columns doesn't match.
Old column names (1): _c0
New column names (448): K-mer, SRR957738, SRR957739, SRR957740, SRR957741, SRR957742, SRR957743, SRR957744, SRR957745, SRR957746, SRR6877286, SRR6877284, SRR6877290, SRR6877288, SRR6877292, SRR6877323, SRR6877304, SRR6877305, SRR6877392, SRR6877308, SRR6877303, SRR6877372, SRR6877370, SRR6877376, SRR6877374, SRR6877378, SRR6877388, SRR6877390, SRR6877384, SRR6877386, SRR6877382, SRR6877354, SRR6877356, SRR6877350, SRR6877352, SRR6877358, SRR6877367, SRR6877365, SRR6877363, SRR6877361, SRR6877369, SRR6877338, SRR6877336, SRR6877334, SRR6877332, SRR6877330, SRR6877342, SRR6877344, SRR6877346, SRR6877348, SRR6877340, SRR6877395, SRR6877397, SRR6877399, SRR6877401, SRR6877403, SRR6877294, SRR6877328, SRR6877339, SRR6877296, SRR6877300, SRR6877316, SRR6877318, SRR6877320, SRR6877322, SRR6877313, SRR6877312, SRR6877310, SRR6877326, SRR6877324, SRR6877282, SRR6877393, SRR6877287, SRR68

In [22]:
aMetagenomes.printSchema()

root
 |-- K-mer: string (nullable = true)
 |-- SRR957738: string (nullable = true)
 |-- SRR957739: string (nullable = true)
 |-- SRR957740: string (nullable = true)
 |-- SRR957741: string (nullable = true)
 |-- SRR957742: string (nullable = true)
 |-- SRR957743: string (nullable = true)
 |-- SRR957744: string (nullable = true)
 |-- SRR957745: string (nullable = true)
 |-- SRR957746: string (nullable = true)
 |-- SRR6877286: string (nullable = true)
 |-- SRR6877284: string (nullable = true)
 |-- SRR6877290: string (nullable = true)
 |-- SRR6877288: string (nullable = true)
 |-- SRR6877292: string (nullable = true)
 |-- SRR6877323: string (nullable = true)
 |-- SRR6877304: string (nullable = true)
 |-- SRR6877305: string (nullable = true)
 |-- SRR6877392: string (nullable = true)
 |-- SRR6877308: string (nullable = true)
 |-- SRR6877303: string (nullable = true)
 |-- SRR6877372: string (nullable = true)
 |-- SRR6877370: string (nullable = true)
 |-- SRR6877376: string (nullable = true)
 

In [23]:
from pyspark.ml.stat import Correlation
from pyspark.ml.stat import Summarizer


In [24]:
aMetagenomes.show()

+--------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--------

In [25]:
aMetagenomesRDD=sc.textFile("kmMatrices/aMetagenomes/whole_aMetagenomes_kmMatrix.txt")

In [30]:
# create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics("mean", "count")

# compute statistics for multiple metrics with weight
aMetagenomes.select(summarizer.summary(aMetagenomes.SRR6877304, aMetagenomes.SRR4047201)).show(truncate=False)



AnalysisException: cannot resolve 'aggregate_metrics(`SRR6877304`, CAST(`SRR4047201` AS DOUBLE))' due to data type mismatch: argument 1 requires vector type, however, '`SRR6877304`' is of string type.;;
'Aggregate [aggregate_metrics(Mean, Count, ComputeMean, ComputeWeightSum, SRR6877304#928, cast(SRR4047201#1124 as double), 0, 0) AS aggregate_metrics(SRR6877304, SRR4047201)#4950]
+- Project [_c0#16 AS K-mer#912, _c1#17 AS SRR957738#913, _c2#18 AS SRR957739#914, _c3#19 AS SRR957740#915, _c4#20 AS SRR957741#916, _c5#21 AS SRR957742#917, _c6#22 AS SRR957743#918, _c7#23 AS SRR957744#919, _c8#24 AS SRR957745#920, _c9#25 AS SRR957746#921, _c10#26 AS SRR6877286#922, _c11#27 AS SRR6877284#923, _c12#28 AS SRR6877290#924, _c13#29 AS SRR6877288#925, _c14#30 AS SRR6877292#926, _c15#31 AS SRR6877323#927, _c16#32 AS SRR6877304#928, _c17#33 AS SRR6877305#929, _c18#34 AS SRR6877392#930, _c19#35 AS SRR6877308#931, _c20#36 AS SRR6877303#932, _c21#37 AS SRR6877372#933, _c22#38 AS SRR6877370#934, _c23#39 